In [16]:
import torch

In [3]:
a = torch.randn((2))

In [4]:
a.shape

torch.Size([2])

In [7]:
a_np = a.data.cpu().numpy()

In [7]:
import numpy as np

In [9]:
np.argmax(a_np)

1

In [11]:
torch.argmax(a).item()

1

In [7]:
indices = torch.randperm(a.shape[0])

In [8]:
indices

tensor([5, 3, 2, 6, 4, 7, 0, 8, 1, 9])

In [4]:
import gym
from collections import deque, namedtuple

In [25]:
model = torch.nn.Sequential(
            torch.nn.Linear(4, 100),
            torch.nn.ReLU(),
            torch.nn.Linear(100, 110),
            torch.nn.ReLU(),
            torch.nn.Linear(110, 2)
        )

In [2]:
env=gym.make("CartPole-v1")

In [3]:
state = env.reset()

array([-0.02816831,  0.01436686,  0.0382335 ,  0.04633472], dtype=float32)

In [11]:
Exp = namedtuple("buffer", field_names=["state", "action", "reward", "done", "next_state"])
replay = deque(maxlen=1000)
batch_size = 10

In [18]:
for i in range(1000):
    state = torch.from_numpy(env.reset())
    next_state = torch.from_numpy(env.reset())
    
    action = np.random.choice(2)
    reward = np.random.choice(2)
    done = np.random.choice(2)
    
    replay.append(Exp(state, action, reward, done, next_state))

In [19]:
random_indices = np.random.choice(len(replay), batch_size, replace=False)

In [20]:
random_indices

array([205, 878, 242, 525, 625, 480, 636, 960, 985, 561])

In [21]:
state_batch = torch.stack([replay[idx].state for idx in random_indices])
next_state_batch = torch.stack([replay[idx].next_state for idx in random_indices])

action_batch = torch.Tensor([replay[idx].action for idx in random_indices])
reward_batch = torch.Tensor([replay[idx].reward for idx in random_indices])
done_batch = torch.Tensor([replay[idx].done for idx in random_indices])

In [22]:
state_batch

tensor([[ 0.0249,  0.0490, -0.0495, -0.0219],
        [-0.0473, -0.0043, -0.0055, -0.0251],
        [-0.0087,  0.0299,  0.0339,  0.0460],
        [ 0.0240,  0.0024, -0.0018, -0.0266],
        [-0.0380,  0.0213, -0.0271, -0.0274],
        [ 0.0487,  0.0271, -0.0224, -0.0051],
        [ 0.0346, -0.0376, -0.0487, -0.0044],
        [ 0.0170, -0.0027, -0.0158,  0.0142],
        [ 0.0027,  0.0031,  0.0209,  0.0150],
        [-0.0495,  0.0387,  0.0492,  0.0480]])

In [27]:
reward_batch + 0.99 * ((1 - done_batch) * torch.max(model(next_state_batch), dim=1)[0])

tensor([8.8088e-03, 1.0000e+00, 1.0040e+00, 1.0000e+00, 0.0000e+00, 3.8696e-04,
        1.5614e-03, 0.0000e+00, 0.0000e+00, 1.0010e-02],
       grad_fn=<AddBackward0>)

In [30]:
model(next_state_batch)

tensor([[-8.5250e-02,  8.8978e-03],
        [-8.2446e-02,  4.2490e-03],
        [-8.3185e-02,  4.0662e-03],
        [-8.7152e-02,  3.3991e-05],
        [-8.5582e-02,  8.6059e-03],
        [-8.6142e-02,  3.9086e-04],
        [-8.7380e-02,  1.5772e-03],
        [-8.8993e-02,  7.4519e-04],
        [-8.2660e-02,  3.1855e-03],
        [-8.6332e-02,  1.0111e-02]], grad_fn=<AddmmBackward0>)

In [36]:
model(state_batch).gather(dim=1, index=action_batch.long().unsqueeze(dim=1)).squeeze()

tensor([-0.0873, -0.0861, -0.0021,  0.0049, -0.0873, -0.0856,  0.0094,  0.0041,
         0.0017, -0.0884], grad_fn=<SqueezeBackward0>)

In [37]:
model(state_batch)

tensor([[-0.0873,  0.0024],
        [-0.0861,  0.0042],
        [-0.0868, -0.0021],
        [-0.0843,  0.0049],
        [-0.0873,  0.0033],
        [-0.0856,  0.0032],
        [-0.0862,  0.0094],
        [-0.0859,  0.0041],
        [-0.0847,  0.0017],
        [-0.0884, -0.0045]], grad_fn=<AddmmBackward0>)

In [41]:
action_batch.int()

tensor([0, 0, 1, 1, 0, 0, 1, 1, 1, 0], dtype=torch.int32)

In [42]:
action_batch.long()

tensor([0, 0, 1, 1, 0, 0, 1, 1, 1, 0])